In [61]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.prompts.prompt import PromptTemplate
from langchain_community.llms import Ollama
from langchain.chains import RetrievalQA
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
import gradio as gr
from langchain_community.document_loaders import WebBaseLoader


In [2]:
# ..............Load the Document.................................
loader = WebBaseLoader(["https://helpcenter.greenestep.com/","https://helpcenter.greenestep.com/documentations/","https://helpcenter.greenestep.com/faq/recently-asked-questions/","https://helpcenter.greenestep.com/knowledgebase/","https://helpcenter.greenestep.com/forums/","https://helpcenter.greenestep.com/resources/"])
documents = loader.load()




In [3]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=30)
docs = text_splitter.split_documents(documents)

In [4]:
# ...............Embeddings........................................
embeddings = OllamaEmbeddings(model="mxbai-embed-large:latest")
index = FAISS.from_documents(docs,embeddings)


In [5]:
docs

[Document(metadata={'source': 'https://helpcenter.greenestep.com/', 'title': 'GreeneStep Help Center – Help Guides, KB Articles, Videos and Forums', 'language': 'en-US'}, page_content='GreeneStep Help Center – Help Guides, KB Articles, Videos and Forums\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\nMenuHome\nFAQs  \n\nFrequently Asked Questions\nProductivity Tips\nTroubleshooting Tips\nGreeneStep Glossary\n\n\nDocumentation  \n\nCRM Cloudfront Portals\nERP Backoffice Suite\nEcommerce Webstore\nTicketing Collaboration\n\nTicketing Collaboration\n\n\nInventory Management\nRetail POS System\n3PL Warehousing Suite\nReports and Dashboards\nGS Extension Plugins\nGIFT BI and Analytics\n\n\nKnowledgebase  \n\nConfiguring GreeneStep\nLogin and Navigation\nSales Orders Management\nEcommerce Management\nRetail Point-Of-Sale\nPurchases and Expenses\nSupply Chain Operations\nFinancial Accounting\nTicketing Collaboration\nOEM Manufacturing\n

In [6]:
# ...............Saving and Loading................................
save_directory = "Storage new wweb"
index.save_local(save_directory)
new_db = FAISS.load_local(save_directory,embeddings,allow_dangerous_deserialization=True)

In [7]:

# ...............Defining Retriever................................
retriever = new_db.as_retriever(search_type="similarity",search_kwargs={"k":3})


In [8]:
# ...............Prompt............................................
template = """ Start answering the question like a chatbot and the question based on the provided context. if you do not found relavent result just say this information is not available in the helpcenter. 

Context: {context}
Question : {question}

Helpful Answer :
"""
prompt = PromptTemplate(template=template,input_variables=["context","question"])


In [9]:
# ..............Define LLM Model....................................
llm = Ollama(model="llama3.1:8b",temperature=0.6)

In [10]:
# ..............Define qa_Chain........................................
llm_chain = LLMChain(
                  llm=llm, 
                  prompt=prompt, 
                  callbacks=None, 
                  verbose=True)


d:\New folder\venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [11]:
document_prompt = PromptTemplate(
    input_variables=["page_content", "source"],
    template="Context:\ncontent:{page_content}\nsource:{source}",
)


In [12]:
combine_documents_chain = StuffDocumentsChain(
                  llm_chain=llm_chain,
                  document_variable_name="context",
                  document_prompt=document_prompt,
                  callbacks=None,
              )


In [13]:
qa = RetrievalQA(
                  combine_documents_chain=combine_documents_chain,
                  verbose=True,
                  retriever=retriever,
                  return_source_documents=True,
              )


d:\New folder\venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `RetrievalQA` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use create_retrieval_chain instead.
  warn_deprecated(


In [14]:
# print(qa("What is described inside?")["result"])

In [60]:
def respond(question,history):
    return qa(question)["result"]


gr.ChatInterface(
    respond,
    chatbot=gr.Chatbot(height=200),
    textbox=gr.Textbox(placeholder="Ask me question related to GES help support", container=False, scale=7),
    title="GES help center",
   
    retry_btn=None,

).launch(share = True)


Running on local URL:  http://127.0.0.1:7877
Running on public URL: https://fc47c9db7657445c87.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)




> Entering new RetrievalQA chain...

> Finished chain.

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
 Start answering the question like a chatbot and the question based on the provided context. if you do not found relavent result just say this information is not available in the helpcenter. 

Context: Context:
content:Great Products				
Liked By All Customers



 


 

				Grow Your Business				
We Help You Grow


 









Our Growing Numbers
True to our motto - Customers for Life! We are doing awesome




HAPPY CLIENTS



SATISFIED USERS



SUPPORT INTERACTIONS



HELP KB ARTICLES












Email: support@greenestep.com | Support hours: 8.00 AM EST to 4.00 PM EST | Call 703-546-4236 Ext 402





About Help Center GreeneStep empowers businesses with next-gen automation solutions to streamline operations and unlock growth, across various industries. Find how GreeneStep can help your business!

Help Center
FAQ
Forums
KnowledgeBase
Documentation
Refe

In [16]:
! pip install gradio_client

In [17]:
from gradio_client import Client

client = Client("https://964fe061a3b85a9383.gradio.live/")
result = client.predict(
		question="Hello!!",
		api_name="/chat"
)
print(result)

Loaded as API: https://964fe061a3b85a9383.gradio.live/ ✔
Yes, our software does support bilingual communication. While it primarily operates in English, it can accept and handle Spanish text. However, there isn't an automatic conversion feature between English and Spanish within the software. But you can customize communications for Spanish-speaking clients by capturing their language preference in the customer master and providing the text in both English and Spanish accordingly. This approach aims to ensure clarity in communications and minimize any misunderstandings or excuses from clients regarding language barriers.


d:\New folder\venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...


> Entering new LLMChain chain...
Prompt after formatting:
 Start answering the question like a chatbot and the question based on the provided context. if you do not found relavent result just say this information is not available in the helpcenter. 

Context: Context:
content:Certainly, we have a feature called Kanban View for sales orders, which allows you to view and schedule deliveries based on the delivery dates specified in the sales orders. This view provides a visual representation of the delivery schedule, making it easier to manage and prioritize orders for delivery. Additionally, we can set up email alerts for delivery schedules if needed.


What is GBS?
#


GreeneStep Business Management Suite (GBS). It refer to complete suite of modules and cloudfront portal. Ecommerce is an add-on module, other extended plug-ins or Warehouse app are not part of the GBS. Contact sales@greenestep.com for more details







FAQ Category Curious to Know